Lightweight CNN model for TI project made to run on Beagle Board. Used to classify Range Doppler Heatmap data into 4 classes: Standing, Sitting, Walking, or Falling.

In [6]:
## All Hyperparameters for Heatmap data
VISUALISE = True  # plot heatmaps for each class
DEMO = 'fall_detection'

# Classes
class_data = {0: 'STANDING', 1: 'SITTING', 2: 'FALLING', 3: 'WALKING'}

# Heatmap dimensions
HEATMAP_HEIGHT = 64  # Range bins
HEATMAP_WIDTH = 32   # Doppler bins

# Training topics
NUM_EPOCHS = 50  # Reduced for initial training
BATCH_SIZE = 16  # Small batch for limited data 
TEST_SIZE_PERCENT = 20
LEARNING_RATE = 0.001  

# Data generation 
SAMPLES_PER_CLASS = 50  

Imports remain the same as TI's pose and fall detection example. Removed CUDA because this model will run on Beagle ARM CPU.

In [7]:
import torch
torch.manual_seed(42)

import torch.nn as nn
import pandas as pd
from torch.optim import Adam 
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import torch.onnx
import sys
from pathlib import Path
import os

print(f"Python version: {sys.version}")
print(f"torch version: {torch.__version__}")

# Device setup
device = "cpu"  # CPU for Beagle compatibility
torch.device(device)
print(f"Device: {device}")

Python version: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
torch version: 2.6.0+cpu
Device: cpu


The following cell loads the heatmap CSV data and prepares it for training (X_list is a numpy array with heatmap data and y_list maintains the classification data "what class")

In [8]:
# Find all CSV files
data_path = Path(os.getcwd() + "/dataset")
class_path = Path(f"{data_path}/classes/")
file_names = [f for f in list(class_path.rglob("*.csv"))]
print(f"Found {len(file_names)} heatmap files")

# Load heatmaps
X_list = []  # List of heatmaps
y_list = []  # List of labels

for file in file_names:
    # Load heatmap (64x32 array)
    heatmap = pd.read_csv(file, header=None).values
    
    # Verify dimensions
    if heatmap.shape != (HEATMAP_HEIGHT, HEATMAP_WIDTH):
        print(f"Warning: {file.name} has shape {heatmap.shape}, expected ({HEATMAP_HEIGHT}, {HEATMAP_WIDTH})")
        continue
    
    # Get classification from folder name
    classification = os.path.basename(os.path.dirname(file)).upper()
    class_id = -1
    for key, val in class_data.items():
        if val == classification:
            class_id = key
            break
    
    if class_id == -1:
        print(f"Warning: Unknown class {classification}")
        continue
    
    # Add to lists
    X_list.append(heatmap)
    y_list.append(class_id)

print(f"\nLoaded {len(X_list)} heatmaps")
print(f"Heatmap shape: {X_list[0].shape}")

Found 0 heatmap files

Loaded 0 heatmaps


IndexError: list index out of range